In [1]:
import ast
import astor
import showast
from pprint import pprint
from packaging import version
from typing import List, Dict

In [2]:
class MyNode(object):
    """
    _file: str = file
        self.ver: version.Version = version.Version(ver)
        with open(file, "r") as f:
            self.file_ast: str = ast.parse(f.read())
        self.name: str = node_name
        self.node: ast.ClassDef = self.node()
        self.methods: Dict[str: list[str]] = {
            node.name: self.get_fn_args(node) 
            for node in self.node.body 
            if isinstance(node, ast.FunctionDef)
        }
        self.meth_names: List[str] = list(self.methods.keys())
    """
    def __init__(self, file: str, ver: str, node_name: str):
        self._file: str = file
        self.ver: version.Version = version.Version(ver)
        with open(file, "r") as f:
            self.file_ast: str = ast.parse(f.read())
        self.name: str = node_name
        self.node: ast.ClassDef = self.node()
        self.methods: Dict[str: list[str]] = {
            node.name: self.get_fn_args(node) 
            for node in self.node.body 
            if isinstance(node, ast.FunctionDef)
        }
        self.meth_names: List[str] = list(self.methods.keys())

    def node(self) -> ast.ClassDef:
        for node in self.file_ast.body:
            if isinstance(node, ast.ClassDef) and node.name == self.name:
                return node
        
    @staticmethod
    def get_fn_args(functionNode: ast.FunctionDef) -> List[str]:
        return [arg.arg for arg in functionNode.args.args]

    def __str__(self) -> str:
        return self.name
    
    def __repr__(self) -> str:
        return f"Node from AST path{self.name}"
    
    def __gt__(self, other_node) -> bool:
        return self.ver > other_node.ver
    
    def __ge__(self, other_node) -> bool:
        return self.ver >= other_node.ver
    
    def __lt__(self, other_node) -> bool:
        return self.ver < other_node.ver
    
    def __le__(self, other_node) -> bool:
        return self.ver <= other_node.ver
    
    def __ne__(self, other_node) -> bool:
        return self.ver != other_node.ver
    
    def __eq__(self, other_node) -> bool:
        return self.ver == other_node.ver
    
    def diff_methods(self, other_node) -> bool:
        older, newer = other_node, self
        if self > other_node:
            older, newer = self, other_node
        
        meths: Dict[str: List[str]] = {}
        meths['added'] = [n for n in newer.meth_names if n not in older.meth_names]
        meths['removed'] = [n for n in older.meth_names if n not in newer.meth_names]
        return meths
        

schema9 = MyNode(file="drf394/rest_framework/schemas/generators.py", ver="3.9.4", node_name="SchemaGenerator")
schema6 = MyNode(file="drf362/rest_framework/schemas.py", ver="3.6.2", node_name="SchemaGenerator")

In [3]:
pprint(schema9.methods)

{'__init__': ['self', 'title', 'url', 'description', 'patterns', 'urlconf'],
 'coerce_path': ['self', 'path', 'method', 'view'],
 'create_view': ['self', 'callback', 'method', 'request'],
 'determine_path_prefix': ['self', 'paths'],
 'get_keys': ['self', 'subpath', 'method', 'view'],
 'get_links': ['self', 'request'],
 'get_schema': ['self', 'request', 'public'],
 'has_view_permissions': ['self', 'path', 'method', 'view']}


In [4]:
pprint(schema6.diff_methods(schema9))

{'added': ['get_link',
           'get_description',
           'get_encoding',
           'get_path_fields',
           'get_serializer_fields',
           'get_pagination_fields',
           'get_filter_fields'],
 'removed': []}
